In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [2]:
# ! pip install "tensorflow>=2.0.0"
# ! pip install --upgrade tensorflow-hub

In [3]:
# ! pip install tensorflow-text

In [11]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

### Adding labels

In [13]:
df['positive'] = df['sentiment'].apply(lambda x:1 if x=='positive' else 0)

In [14]:
df.head()

,review,sentiment,positive
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['review'], df['positive'], stratify=df['positive'])

### Getting started with BERT

In [16]:
# Downloading the BERT model

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [17]:
# Initializing the BERT layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [20]:
# Initializing the neural network layers

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [22]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

### Model compling

In [23]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

### Fitting the model

In [24]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1172/1172 [==============================] - 222s 181ms/step - loss: 0.6092 - accuracy: 0.6662 - precision: 0.6701 - recall: 0.6548
Epoch 2/5
1172/1172 [==============================] - 212s 181ms/step - loss: 0.5524 - accuracy: 0.7241 - precision: 0.7280 - recall: 0.7156
Epoch 3/5
1172/1172 [==============================] - 212s 181ms/step - loss: 0.5335 - accuracy: 0.7348 - precision: 0.7403 - recall: 0.7233
Epoch 4/5
1172/1172 [==============================] - 209s 179ms/step - loss: 0.5253 - accuracy: 0.7414 - precision: 0.7462 - recall: 0.7317
Epoch 5/5
1172/1172 [==============================] - 213s 182ms/step - loss: 0.5189 - accuracy: 0.7445 - precision: 0.7501 - recall: 0.7332


### Evaluating the model

In [25]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

391/391 [==============================] - 71s 180ms/step


### References
https://www.section.io/engineering-education/classification-model-using-bert-and-tensorflow/